In [4]:
import cv2
import numpy as np



ModuleNotFoundError: No module named 'dlib'

In [1]:
import cv2
import numpy as np

# Load facial classifiers
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Read the uploaded cat filter image
cat = cv2.imread('pngegg.png', -1)  # -1 to read with the alpha channel

# Check if the image has an alpha channel
if cat.shape[2] == 4:
    # Split the channels
    b_channel, g_channel, r_channel, alpha_channel = cv2.split(cat)
    
    # Create a mask and inverse mask using the alpha channel
    original_mask = alpha_channel
    original_mask_inv = cv2.bitwise_not(original_mask)
    
    # Convert the image to BGR (dropping alpha channel)
    cat = cv2.merge((b_channel, g_channel, r_channel))
else:
    # Convert to grayscale and create a binary mask if no alpha channel
    cat_gray = cv2.cvtColor(cat, cv2.COLOR_BGR2GRAY)
    _, original_mask = cv2.threshold(cat_gray, 10, 255, cv2.THRESH_BINARY_INV)
    original_mask_inv = cv2.bitwise_not(original_mask)

# Video capture
cap = cv2.VideoCapture(0)

while True:
    # Read each frame of video and convert to gray
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_h, img_w = img.shape[:2]

    # Find faces in the image using classifier
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        # Resize the cat filter to fit on the face
        cat_width = w
        cat_height = int(cat_width * cat.shape[0] / cat.shape[1])
        cat = cv2.resize(cat, (cat_width, cat_height), interpolation=cv2.INTER_AREA)
        mask = cv2.resize(original_mask, (cat_width, cat_height), interpolation=cv2.INTER_AREA)
        mask_inv = cv2.resize(original_mask_inv, (cat_width, cat_height), interpolation=cv2.INTER_AREA)

        # Define the region of interest on the face
        roi = img[y:y+cat_height, x:x+cat_width]

        # Use the mask to create the region of the cat filter
        img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
        cat_fg = cv2.bitwise_and(cat, cat, mask=mask)

        # Combine the region with the cat filter
        dst = cv2.add(img_bg, cat_fg)
        img[y:y+cat_height, x:x+cat_width] = dst

    # Display the resulting frame
    cv2.imshow('img', img)

    # Break the loop with the 'q' key
    if cv2.waitKey(1) == ord('q'):
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()